In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [2]:
load_dotenv()

llm = ChatGroq(model='Gemma2-9b-it')

In [3]:
class JokeState(TypedDict):
    topic: str
    joke: str
    explanation:str

In [5]:
def generate_joke(state: JokeState):

    prompt = f"generate a joke on the topic {state['topic']}"
    response = llm.invoke(prompt).content

    return {'joke': response}

In [7]:
def generate_explanation(state: JokeState):

    prompt = f"write an explanation for the joke - {state['joke']}"
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [9]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke',generate_joke)
graph.add_node('generate_explanation',generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke','generate_explanation')
graph.add_edge('generate_explanation',END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [10]:
config1= {'configurable':{'thread_id':'1'}}

workflow.invoke({'topic':'pizza'},config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza slice go to the doctor? \n\nBecause it was feeling a little cheesy! 🍕  \n',
 'explanation': 'This is a play on words!  \n\n* **Cheesy** can refer to something being cheesy in the sense of being over-the-top or corny. \n* **Cheesy** can also refer to the type of dairy product used on pizza.\n\nThe joke makes a humorous connection between these two meanings. The pizza slice is feeling "a little cheesy" in the sense that it\'s feeling unwell, but it also literally *is* cheesy because it\'s made of cheese.  \n\n'}

In [11]:
# get last state

workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza slice go to the doctor? \n\nBecause it was feeling a little cheesy! 🍕  \n', 'explanation': 'This is a play on words!  \n\n* **Cheesy** can refer to something being cheesy in the sense of being over-the-top or corny. \n* **Cheesy** can also refer to the type of dairy product used on pizza.\n\nThe joke makes a humorous connection between these two meanings. The pizza slice is feeling "a little cheesy" in the sense that it\'s feeling unwell, but it also literally *is* cheesy because it\'s made of cheese.  \n\n'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07339a-a332-6196-8002-f6acae7c12c9'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-07T02:53:15.429314+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07339a-9d19-6624-8001-7943e757d919'}}, tasks=(), interrupts=())

In [13]:
# get intermediate state

list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza slice go to the doctor? \n\nBecause it was feeling a little cheesy! 🍕  \n', 'explanation': 'This is a play on words!  \n\n* **Cheesy** can refer to something being cheesy in the sense of being over-the-top or corny. \n* **Cheesy** can also refer to the type of dairy product used on pizza.\n\nThe joke makes a humorous connection between these two meanings. The pizza slice is feeling "a little cheesy" in the sense that it\'s feeling unwell, but it also literally *is* cheesy because it\'s made of cheese.  \n\n'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07339a-a332-6196-8002-f6acae7c12c9'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-07T02:53:15.429314+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07339a-9d19-6624-8001-7943e757d919'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'to

Time Travel

In [ ]:
# get a particular checkpointer

workflow.get_state({'configurable':{'thread_id':'1','checkpoint_id':'1f07339a-9d19-6624-8001-7943e757d919'}})

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza slice go to the doctor? \n\nBecause it was feeling a little cheesy! 🍕  \n'}, next=('generate_explanation',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f07339a-9d19-6624-8001-7943e757d919'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-08-07T02:53:14.790045+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07339a-8f86-697b-8000-e29a5065c627'}}, tasks=(PregelTask(id='ccdf6e5a-8e22-a7d9-da7a-4afd3f90ca1a', name='generate_explanation', path=('__pregel_pull', 'generate_explanation'), error=None, interrupts=(), state=None, result={'explanation': 'This is a play on words!  \n\n* **Cheesy** can refer to something being cheesy in the sense of being over-the-top or corny. \n* **Cheesy** can also refer to the type of dairy product used on pizza.\n\nThe joke makes a humorous connection between these two meanings. The pizza slice is feel

In [15]:
workflow.invoke(None, {'configurable':{'thread_id':'1','checkpoint_id':'1f07339a-9d19-6624-8001-7943e757d919'}})

{'topic': 'pizza',
 'joke': 'Why did the pizza slice go to the doctor? \n\nBecause it was feeling a little cheesy! 🍕  \n',
 'explanation': 'This joke plays on the word "cheesy."  \n\n* **Literal Meaning:**  Pizza is made with cheese, so a pizza slice literally *is* cheesy.\n* **Figurative Meaning:** "Cheesy" can also mean something that is corny, cheesy, or unoriginal. \n\nThe joke sets up the expectation that the pizza slice is sick in a real way, but then uses the word "cheesy" in its figurative sense to make a funny, unexpected connection to the pizza\'s ingredients.  \n\n'}

In [16]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza slice go to the doctor? \n\nBecause it was feeling a little cheesy! 🍕  \n', 'explanation': 'This joke plays on the word "cheesy."  \n\n* **Literal Meaning:**  Pizza is made with cheese, so a pizza slice literally *is* cheesy.\n* **Figurative Meaning:** "Cheesy" can also mean something that is corny, cheesy, or unoriginal. \n\nThe joke sets up the expectation that the pizza slice is sick in a real way, but then uses the word "cheesy" in its figurative sense to make a funny, unexpected connection to the pizza\'s ingredients.  \n\n'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0733ac-d269-6717-8002-f63a6e81e29d'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-07T03:01:23.564111+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07339a-9d19-6624-8001-7943e757d919'}}, tasks=(), interrupts=()),
 Sta

Updating State

In [17]:
workflow.update_state({'configurable':{'thread_id':'1','checkpoint_id':'1f07339a-9d19-6624-8001-7943e757d919','checkpoint_ns':'' }},{'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0733bb-3ee1-61a8-8002-5dab204db0d8'}}

In [18]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the pizza slice go to the doctor? \n\nBecause it was feeling a little cheesy! 🍕  \n'}, next=('generate_explanation',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0733bb-3ee1-61a8-8002-5dab204db0d8'}}, metadata={'source': 'update', 'step': 2, 'parents': {}}, created_at='2025-08-07T03:07:50.747383+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07339a-9d19-6624-8001-7943e757d919'}}, tasks=(PregelTask(id='d0e3187a-b188-e82f-60c1-868c256b8a16', name='generate_explanation', path=('__pregel_pull', 'generate_explanation'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza slice go to the doctor? \n\nBecause it was feeling a little cheesy! 🍕  \n', 'explanation': 'This joke plays on the word "cheesy."  \n\n* **Literal Meaning:**  Pizza is made with cheese, so

In [23]:
workflow.invoke(None, {'configurable':{'thread_id':'1','checkpoint_id':'1f0733bb-3ee1-61a8-8002-5dab204db0d8'}})

{'topic': 'samosa',
 'joke': 'Why did the pizza slice go to the doctor? \n\nBecause it was feeling a little cheesy! 🍕  \n',
 'explanation': 'This joke plays on the double meaning of the word "cheesy."\n\n* **Literal meaning:**  Pizza is made with cheese, so a slice of pizza literally *is* cheesy.\n\n* **Figurative meaning:**  "Cheesy" can also mean something that is corny, unoriginal, or cheesy as in lacking sophistication. \n\nThe joke works because it sets up the expectation of a serious reason for a pizza slice to go to the doctor, but then delivers a silly punchline based on the playful use of the word "cheesy."  \n\n'}